# Read All Dataset CSV

In [1]:
import os
import csv
import pandas as pd
import numpy as np

In [18]:
dataset_names=[]
X_trains=[]
y_trains=[]
X_tests=[]
for folder_name in os.listdir("./Competition_data"):
    # print(folder_name)
    dataset_names.append(folder_name)
    X_trains.append(pd.read_csv(f"./Competition_data/{folder_name}/X_train.csv",header=0))
    y_trains.append(pd.read_csv(f"./Competition_data/{folder_name}/y_train.csv",header=0))
    X_tests.append(pd.read_csv(f"./Competition_data/{folder_name}/X_test.csv",header=0))

## Data Preprocessing & Feature Engineering

In [19]:
## your code here

## train test split & build Model
You can select an appropriate model and perform corresponding hyperparameter tuning.

In [20]:
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from imblearn.over_sampling import SMOTE
from catboost import CatBoostClassifier, Pool  
import lightgbm as lgb
import torch
from pytorch_tabnet.tab_model import TabNetClassifier
# from tab_transformer import TabTransformer
# from sklearn.metrics import accuracy_score, f1_score, precision_score
from sklearn.metrics import roc_auc_score

In [21]:
models=[]
for i in range(len(dataset_names)):
    tmp_X_train, tmp_X_test, tmp_y_train, tmp_y_test = train_test_split(X_trains[i], y_trains[i], test_size=0.2, random_state=42)
    # 初始化 TabNet 模型
    # model = TabNetClassifier()

    # 訓練模型
    # model.fit(X_trains[i].values, y_trains[i].values.squeeze())
    # model = KNeighborsClassifier(n_neighbors=3)
    # model = RandomForestClassifier(n_estimators=200, max_depth=3, random_state=0)
    # model = RandomForestClassifier(n_estimators=200, max_depth=3, random_state=0, class_weight='balanced')
    # 計算類別比例，用於 scale_pos_weight
    pos_weight = (y_trains[i] == 0).sum() / (y_trains[i] == 1).sum()
    # 確保 pos_weight 是 float 類型
    pos_weight = float(pos_weight.iloc[0])  # 使用 .iloc[0] 來提取第一個元素
    # 使用 XGBoost 並設置 scale_pos_weight
    # model = XGBClassifier(n_estimators=200, max_depth=3, scale_pos_weight=pos_weight, random_state=0, use_label_encoder=False, eval_metric='logloss')
    # model = XGBClassifier(
    #     n_estimators=200,
    #     max_depth=3,
    #     scale_pos_weight=pos_weight,
    #     random_state=0,
    #     use_label_encoder=False,
    #     eval_metric='logloss',
    #     subsample=0.8,  # 隨機抽取80%的樣本來建樹
    #     colsample_bytree=0.8,  # 每棵樹隨機選擇80%的特徵
    #     reg_alpha=0.1,  # L1正則化
    #     reg_lambda=1.0  # L2正則化 
    # )
    # cat_features = X_trains[i].select_dtypes(include=['object', 'category']).columns.tolist()

    # 將類別型特徵的名稱轉換為對應的索引
    # cat_feature_indices = [X_trains[i].columns.get_loc(col) for col in cat_features]
    # Using CatBoostClassifier
    # model = CatBoostClassifier(
    #     iterations=200,
    #     depth=3,
    #     learning_rate=0.1,
    #     scale_pos_weight=pos_weight,  # Helpful for imbalanced datasets
    #     random_state=0,
    #     cat_features=cat_feature_indices,  # Specify categorical feature indices if available
    #     verbose=0
    # )
    # 使用 SMOTE 增加少數類別樣本
    # smote = SMOTE(random_state=42)
    # X_train_res, y_train_res = smote.fit_resample(X_trains[i], y_trains[i])
    # model = CatBoostClassifier(
    #     iterations=200,
    #     learning_rate=0.1,
    #     depth=3,
    #     l2_leaf_reg=3,  # 正則化參數
    #     random_state=42,
    #     cat_features=cat_feature_indices,  # Specify categorical feature indices if available
    #     # scale_pos_weight=pos_weight,  # Helpful for imbalanced datasets
    #     verbose=0,
    #     thread_count=-1
    # )

    model = lgb.LGBMClassifier(
        n_estimators=500,
        max_depth=3,
        scale_pos_weight=pos_weight,  # 處理類別不平衡
        random_state=0,
        class_weight='balanced',  # 或使用class_weight來處理類別不平衡
        is_unbalanced=True,  # 開啟不平衡資料自動處理
        learning_rate=0.05,
        lambda_l1=0.1,  # L1正則化
        lambda_l2=1.0   # L2正則化
    )

    # # 使用 TabNet 並設置適當的超參數
    #  # 使用 TabNet 並設置適當的超參數
    # model = TabNetClassifier(
    #     n_d=8,  # Encoder layer's dimensionality
    #     n_a=8,  # Attention block's dimensionality
    #     n_steps=3,  # Number of steps in the architecture
    #     gamma=1.5,  # Coefficient of attention loss
    #     lambda_sparse=1e-5,  # Sparse regularization
    #     optimizer_fn=torch.optim.Adam,  # Optimizer
    #     optimizer_params=dict(lr=2e-2),  # Learning rate
    #     mask_type='entmax',  # Mask type, can also be 'sparsemax'
    # )

    # # 使用 .values 轉換為 NumPy 陣列
    # model.fit(
    #     X_trains[i].values,  # 將訓練集轉為 NumPy
    #     y_trains[i].values.squeeze(),  # 目標變數轉換為一維數組
    #     eval_set=[(tmp_X_test.values, tmp_y_test.values.squeeze())],  # 將驗證集轉為 NumPy
    #     batch_size=1024, 
    #     virtual_batch_size=128,
    #     max_epochs=100,
    #     patience=20,
    # )
    model.fit(X_trains[i], y_trains[i].squeeze())
    # model.fit(X_trains[i], y_trains[i].squeeze())
    # tmp_y_prob = model.predict_proba(tmp_X_test)[:, 1]
    # auc = roc_auc_score(tmp_y_test, tmp_y_prob)
    models.append(model)



[LightGBM] [Warning] Unknown parameter: is_unbalanced
[LightGBM] [Warning] lambda_l2 is set=1.0, reg_lambda=0.0 will be ignored. Current value: lambda_l2=1.0
[LightGBM] [Warning] lambda_l1 is set=0.1, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.1
[LightGBM] [Warning] Unknown parameter: is_unbalanced
[LightGBM] [Warning] lambda_l2 is set=1.0, reg_lambda=0.0 will be ignored. Current value: lambda_l2=1.0
[LightGBM] [Warning] lambda_l1 is set=0.1, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.1
[LightGBM] [Info] Number of positive: 249, number of negative: 100
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000348 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 476
[LightGBM] [Info] Number of data points in the train set: 349, number of used features: 11
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Info] Start training from score 0.

## Inference Model

In [22]:
y_predicts=[]
for i in range(len(dataset_names)):
    y_predict_proba=models[i].predict_proba(X_tests[i])[:, 1]
    df = pd.DataFrame(y_predict_proba, columns=['y_predict_proba'])
    y_predicts.append(df)
    

[LightGBM] [Warning] Unknown parameter: is_unbalanced
[LightGBM] [Warning] lambda_l2 is set=1.0, reg_lambda=0.0 will be ignored. Current value: lambda_l2=1.0
[LightGBM] [Warning] lambda_l1 is set=0.1, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.1
[LightGBM] [Warning] Unknown parameter: is_unbalanced
[LightGBM] [Warning] lambda_l2 is set=1.0, reg_lambda=0.0 will be ignored. Current value: lambda_l2=1.0
[LightGBM] [Warning] lambda_l1 is set=0.1, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.1
[LightGBM] [Warning] Unknown parameter: is_unbalanced
[LightGBM] [Warning] lambda_l2 is set=1.0, reg_lambda=0.0 will be ignored. Current value: lambda_l2=1.0
[LightGBM] [Warning] lambda_l1 is set=0.1, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.1
[LightGBM] [Warning] Unknown parameter: is_unbalanced
[LightGBM] [Warning] lambda_l2 is set=1.0, reg_lambda=0.0 will be ignored. Current value: lambda_l2=1.0
[LightGBM] [Warning] lambda_l1 is set=0.1, reg_alpha=0.0 wi

## Save result

In [23]:
for idx,dataset_name in enumerate(dataset_names):
    df=y_predicts[idx]
    df.to_csv(f'./Competition_data/{dataset_name}/y_predict.csv', index=False,header=True)